In [1]:
from sklearn import metrics
import pandas as pd
from torch import nn
from pyarabic.araby import tokenize
import numpy as np
import pickle
import spacy
import torch

# from model_building import Classifier 
from pre_processing_post import processPost
from feature_extraction import get_ngram_features, get_word_embedding_features, avg_word_vector
from gru_model import ArabicDataset, Classifier, evaluate, train

In [2]:
# needed functions
def print_report(y_pred, y_test):
    report = metrics.classification_report(y_test, y_pred)
    print(report)
    print("accuracy: {:0.3f}".format(metrics.accuracy_score(y_test, y_pred)))

In [3]:
# load AraVec Spacy model
nlp = spacy.load("./spacy.aravec.model/")

c:\Users\Nada\anaconda3\lib\site-packages\spacy\util.py:837: UserWarning: [W095] Model 'ar_pipeline' (0.0.0) was trained with spaCy v3.4 and may not be 100% compatible with the current version (3.3.1). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


In [4]:
test_data = pd.read_csv('./DataSet/dev.csv',sep=',',header=0)
test_data['text'] = test_data['text'].apply(lambda x: processPost(x))

In [5]:
# load word2index dictionary
with open('./vocab/stance/word2index.pickle', 'rb') as f:
    word2index = pickle.load(f)

In [6]:
weights_train_matrix = []
for word in word2index:
  weights_train_matrix.append(nlp(word).vector)

weights_train_matrix = torch.from_numpy(np.array(weights_train_matrix))
weights_train_matrix.size()

torch.Size([12538, 100])

td-idf feature

In [7]:
# load naive bayes model
with open('./models/stance/NaiveBayes_tfidf.sav', 'rb') as f:
    naive_bayes_model = pickle.load(f)

with open('./models/stance/TFIDFVectorizer.sav', 'rb') as f:
    word_vectorizer = pickle.load(f)


X_test_tfidf = word_vectorizer.transform(test_data['text'])
y_pred = naive_bayes_model.predict_proba(X_test_tfidf)

BoW feature

In [8]:
# load complement NB model
with open('./models/stance/BoWVectorizer.sav', 'rb') as f:
    word_vectorizer_BoW = pickle.load(f)
with open('./models/stance/NaiveBayes_BoW.sav', 'rb') as f:
    naive_bayes_model = pickle.load(f)

X_test_Bow = word_vectorizer.transform(test_data['text'])
y_pred += naive_bayes_model.predict_proba(X_test_Bow)

In [9]:
#load GRU model
with open('./models/stance/GRU_Ara2Vec.pth', 'rb') as f:
    gru_model = Classifier(weights_train_matrix)
    gru_model.load_state_dict(torch.load(f)) 
    gru_model.eval()

test_data_tokenized = test_data['text'].apply(tokenize)
test_data_tokenized_as_num = test_data_tokenized.apply(lambda x: [word2index[word] for word in x if word in word2index])
test_dataset = ArabicDataset(list(test_data_tokenized_as_num), test_data['stance'] + 1, word2index['<فراغ>'])
y_pred += evaluate(gru_model,test_dataset)

100%|██████████| 32/32 [00:01<00:00, 22.83it/s]


accuracy: 0.770
              precision    recall  f1-score   support

           0       0.28      0.30      0.29        70
           1       0.38      0.54      0.45       126
           2       0.91      0.85      0.88       804

    accuracy                           0.77      1000
   macro avg       0.52      0.56      0.54      1000
weighted avg       0.80      0.77      0.78      1000


Test Accuracy: 0.7699999809265137


In [43]:
# # load transformer probas
# with open('./Transformer/y_prob_stance.pickle', 'rb') as f:
#     y_prob_stance = pickle.load(f)
# y_pred += y_prob_stance

In [10]:
y_pred /= 3
y_pred = np.argmax(y_pred, axis=1) - 1
print_report(y_pred, test_data['stance'])

              precision    recall  f1-score   support

          -1       0.28      0.31      0.30        70
           0       0.39      0.56      0.46       126
           1       0.92      0.85      0.88       804

    accuracy                           0.77      1000
   macro avg       0.53      0.57      0.55      1000
weighted avg       0.81      0.77      0.79      1000

accuracy: 0.773
